# Standardization

Based on [DataRescaling.ipynb.ipynb from LMU course](https://github.com/fuenfundachtzig/LMU_DA_ML_Basic/blob/main/notebooks/DataRescaling.ipynb)

Most features in our data are just numbers without units attached to them. However, giving a length in cm or inch (or in [Newton seconds vs pound-force seconds][1]) will give different values and thus different outputs when feeding these numbers e.g. to a neural network. 

---
### Mars Climate Orbiter: [a sad story](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter#Cause_of_failure)
* robotic space probe, mission launched Dec 98, planned to reach Mars orbit in Sep 99
* Trajectory Correction Maneuver-4 was computed beginning of September
    * some ground software by Lockheed Martin produced results in a United States customary unit, contrary to its specification, while a second system by NASA expected those results to be in SI units
    * specifically: the total impulse produced by thruster firings was given in pound-force seconds instead of Ns (factor 4.45 mismatch)
* unfortunately, the disagreement was noticed but the concern by the engineers ignored because they didn't fill the forms correctly
    * a 5th correction maneuver was considered but not done
* the probe entered a too low Mars orbit (57 km instead of 226 km)
    * communication was lost on 23.09.1999 when it went into orbital insertion
    * likely skipped from the atmosphere and reentered space or was destroyed
* total cost of the mission: around 0.33 billion USD

---

Even more importantly, the values of different features may naturally cover completely different ranges. This may have a large effect on the performance of machine learning models:
* obviously e.g. when computing the distance of nearest neighbors in the kNN estimator based on several features
* but also if a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly.
* rescaling features to have mean 0 and standard deviation 1 effectively gets rid of the units: instead, we measure in “standard deviations from the mean”

In general, learning algorithms benefit from standardization of the data set.
Some algorithms will be more susceptible to the scaling of the data than others:
* Neural networks expect all input features to vary in a similar way, and ideally to look like standard normally distributed data, i.e. Gaussian with zero mean and unit variance.
* Decision Trees and BDTs just apply `if` statements on the features and are typically very robust.

The `sklearn` documentation provides further information on [preprocessing of data][2].

[1]: https://en.wikipedia.org/wiki/Mars_Climate_Orbiter#Cause_of_failure
[2]: https://scikit-learn.org/stable/modules/preprocessing.html

<div class="alert alert-block alert-success">
    <h3>Exercise</h3>
    Read about different <a href="https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#original-data">scalers in <code>sklearn</code></a>.
</div>

### Example I
One example that illustrates the importance of preprocessing the data to rescale feature values to "standard ranges" is the application of a *support vector machine* (SVM) to `sklearn`'s cancer dataset.

Linear support vector classifiers (SVC) try to find a (hyper-) plane in phase space that optimally separates the two given populations. 
* Predictions are then made by checking on which side of this plane new samples lie. 
* The (hyper-) plane is defined by only a few of the training samples (the supporting vectors of features); those that lie close to the resulting plane.
* note: distinction of *generative* classification vs *discriminative* classification: 
  * generative classification: model each class and compute likelihood of belonging to each (e.g. Gaussian Naive Bayes)
  * discriminative classification: find curve or manifold in feature space separating the classes from each other

We use here a SVM implemented in `sklearn.svm.SVC`. This is (by default) not a linear SVC but a kernelized SVC which can yield a more complex model than a simple hyperplanes in the input space.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=0
)

The features in this dataset have very different ranges as illustrated by the following plot.

In [ ]:
plt.figure(figsize=(10, 4))
plt.boxplot(cancer.data, showfliers=False) # boxes: lower to upper quartile, whiskers: Q1-1.5IQR/Q3+1.5IQR, outliers are not shown
plt.xlabel("feature")
plt.yscale("log")

In [ ]:
# fit data without rescaling
svm = SVC(gamma="auto")
svm.fit(X_train, y_train).score(X_test, y_test)

This is a very low accuracy compared to our earlier results.

(Note that `SVC` by default uses `kernel="rbf"`, i.e. a [radial-basis-function kernel][1]. The use of the Euclidean distance between the feature vectors may be causing this. If we instead used a linear SVC (`SVC(kernel="linear")`), the resulting score would be much higher.)

[1]: https://en.wikipedia.org/wiki/Radial_basis_function_kernel

In [ ]:
# now rescale the features and retrain
from sklearn.preprocessing import MinMaxScaler
# compute minimum and maximum on the training data
# note that we must train the scaler only on the training data (but not the full dataset)
scaler = MinMaxScaler().fit(X_train)
# rescale the training data
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)
# retrain
svm.fit(X_train_scaled, y_train).score(X_test_scaled, y_test)

In [ ]:
plt.figure(figsize = (10, 4))
plt.boxplot(scaler.transform(cancer.data), showfliers = False);

In [ ]:
# same effect (although much smaller) e.g. on MLP
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=1000, random_state=0)
print("Score using unscaled data:", mlp.fit(X_train, y_train).score(X_test, y_test))
print("Score using rescaled data:", mlp.fit(X_train_scaled, y_train).score(X_test_scaled, y_test))

In [ ]:
# no effect e.g. on BDT
from sklearn.ensemble import GradientBoostingClassifier
bdt = GradientBoostingClassifier(random_state=0)
print("Score using unscaled data:", bdt.fit(X_train, y_train).score(X_test, y_test))
print("Score using rescaled data:", bdt.fit(X_train_scaled, y_train).score(X_test_scaled, y_test))

### Example II

Large values of features may also lead to convergence issues. 
(Note that many linear models include regularization terms that are introduced to avoid very large values of the parameters.) 

Let us write a quick function to define a dataset with two populations that should be easy to separate:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

def make_line(a, b, noise, nsamples):
    x = np.random.rand(nsamples)
    return np.stack([x, a * x + b + (np.random.rand(nsamples) - 0.5) * noise], axis=1)

def make_xy_corr_sample(noise=0.3, nsamples=50, xscale=1, yscale=1e5):
    X = np.concatenate(
        [
            make_line(0.5, 0, noise=noise, nsamples=nsamples),
            make_line(0.5, 0.5, noise=noise, nsamples=nsamples),
        ],
    )
    X[:, 0] *= xscale
    X[:, 1] *= yscale
    y = np.concatenate([np.zeros(nsamples), np.ones(nsamples)])
    return X, y

X, y = make_xy_corr_sample()
plt.scatter(*X[y==0].T)
plt.scatter(*X[y==1].T)

Plot: We have two populations that are obviously easy to separate? Or are they?

The two populations should be easy to separate with a line, so we will try a linear SVC in this case.

In [ ]:
### prepare data
from sklearn.model_selection import train_test_split
X, y = make_xy_corr_sample()
X_train, X_test, y_train, y_test = train_test_split(X, y)

### fit
from sklearn.svm import LinearSVC

model = LinearSVC().fit(X_train, y_train)

print("Score on training data:", model.score(X_train, y_train))
print("Score on validation data:", model.score(X_test, y_test))

The score on the training data shows that we cannot even model the training data correctly using the linear model.

In [ ]:
def vis_classifier(model, xmin, xmax, ymin, ymax, **kwargs):
    xx, yy = np.meshgrid(
        np.linspace(xmin, xmax, 100),
        np.linspace(ymin, ymax, 100),
    )
    X = np.stack([xx, yy], axis=-1).reshape(-1, 2)
    zz = model.predict(X).reshape(xx.shape)
    plt.pcolormesh(xx, yy, zz, **kwargs)


vis_classifier(
    model, X[:, 0].min(), X[:, 0].max(), X[:, 1].min(), X[:, 1].max(), cmap="RdBu_r", alpha=0.8
)
plt.scatter(*X[y == 0].T, color="blue")
plt.scatter(*X[y == 1].T, color="red")

<div class="alert alert-block alert-success">
    <h3>Exercise 2</h3>
    How can we solve this problem?
</div>